In [1]:
import polars as pl
import duckdb
from core_pro.ultilities import make_sync_folder
from create_index import create_search_result

In [2]:
path = make_sync_folder("dataset/item_matching")
file = path / "data_sample_FMCG_clean.parquet"

query = f"""
select item_id
, item_name
from read_parquet('{file}')
"""
df = duckdb.sql(query).pl()
df.head()

item_id,item_name
i64,str
21740433922,"""Băng vệ sinh cho mẹ sau sinh M…"
22380668240,"""(tặng quà) Vitamin D3 K2 Bioam…"
12552302079,"""Kẹo thập cẩm dẻo đặc sản Langf…"
28355345290,"""Trà thảo mộc detox HERA MATE c…"
15130518417,"""Ngũ Cốc Mè Đen Damtuh Hàn Quốc…"


In [3]:
file_embed = path / "bgem3_compress_embeds.npy"
df_match = create_search_result(path=path, file_embed=file_embed, data=df)

(78265, 512)


  0%|                                                                   | 0/1 [00:00<?, ?it/s]


In [4]:
file_embed = path / "bgem3_embeds.npy"
df_match_bge = create_search_result(path=path, file_embed=file_embed, data=df)

(78265, 1024)


  0%|                                                                   | 0/1 [00:00<?, ?it/s]


In [5]:
dict_bgecom = df_match.to_dicts()
dict_bge = df_match_bge.to_dicts()

In [10]:
df_match.head()

item_id,item_name,embed,db_item_id,db_item_name,score
i64,str,"array[f32, 512]",list[i64],list[str],list[f32]
21740433922,"""Băng vệ sinh cho mẹ sau sinh M…","[-0.052929, 0.019345, … -0.000745]","[21740433922, 28471465187, … 19891380892]","[""Băng vệ sinh cho mẹ sau sinh MAMA"", ""Kẹp Phồng Chân Tóc Tự Dính,Uốn Mái Thưa Phong Cách Hàn Quốc - KAYFIA"", … ""Sữa Tắm Giảm Mụn Lưng, Viêm Nang Lông Giúp Cải Thiện Da Sau 2 Tuần Sử Dụng- 210ml""]","[1.0, 0.96463, … 0.712332]"
22380668240,"""(tặng quà) Vitamin D3 K2 Bioam…","[0.106106, -0.04425, … -0.013286]","[22380668240, 27408617454, … 21586376516]","[""(tặng quà) Vitamin D3 K2 Bioamicus 10ml giúp bé phát triển chiều cao"", ""Bột Diếp Cá Nguyên Chất Ngừa Mụn, Giảm Các Vết Thâm Và Thanh Nhiệt Cơ Thể Túi 100g"", … ""Mặt Nạ BANOBAGI Cấp Ẩm Da Căng Bóng Super Collagen Mask 30ml - SUPER Aqua""]","[1.0, 0.737776, … 0.700294]"
12552302079,"""Kẹo thập cẩm dẻo đặc sản Langf…","[0.089142, -0.35271, … -0.001693]","[12552302079, 22250090102, … 20077914840]","[""Kẹo thập cẩm dẻo đặc sản Langfarm - Thơm ngon, ít ngọt, an toàn vệ sinh, nguồn gốc rõ ràng"", ""Kính áp tròng Lan Hạ màu nâu cam sáng trong veo Hàn Quốc hãng Dabae"", … ""Sáp Ngăn Mùi NIVEA MEN Phân Tử Bạc (50 ml) - 83181""]","[1.0, 0.972615, … 0.769207]"
28355345290,"""Trà thảo mộc detox HERA MATE c…","[0.230578, -0.454291, … -0.008817]","[28355345290, 4537529012, … 25363162363]","[""Trà thảo mộc detox HERA MATE chính hãng hộp 30 gói"", ""NAKIZ LIVELY BUTT 〡 Kem thoa thâm, mụn Mông, Bẹn, Nách, Lưng, Nhũ hoa, Cùi chỏ, Đầu gối .... Thái Lan"", … ""Combo tắm gội dưỡng sinh Wonmom ( 1 Dầu Gội Cốt Bồ Kết + 1 Sữa tắm Cho Mẹ + 1 Dung Dịch Vệ Sinh )""]","[1.0, 0.901268, … 0.888879]"
15130518417,"""Ngũ Cốc Mè Đen Damtuh Hàn Quốc…","[0.123364, -0.376138, … -0.012691]","[15130518417, 8689421259, … 20268690513]","[""Ngũ Cốc Mè Đen Damtuh Hàn Quốc - Hộp 40 Gói"", ""Kẹo mút Chupa Chups hình dáng Bó hoa hồng hương trái cây (19 que kẹo) phù hợp tặng người thương nhân dịp đặc biệt"", … ""Kem chống nắng Heliocare 360° Water Gel 50ml SPF 50+ cho mọi loại da""]","[1.0, 0.987829, … 0.825651]"


In [24]:
idx = 69
print(f"Orginal: {dict_bge[idx]["item_name"]}")
for i, s in zip(dict_bge[idx]["db_item_name"], dict_bge[idx]["score"]):
    print(f"- {i} -- {s:,.3f}")
print("====")
for i, s in zip(dict_bgecom[idx]["db_item_name"], dict_bgecom[idx]["score"]):
    print(f"- {i} -- {s:,.3f}")

Orginal: Sữa Tăng Cân Cho Trẻ Em Appeton Weight Gain Junior hương Vanila 450g
- Sữa Tăng Cân Cho Trẻ Em Appeton Weight Gain Junior hương Vanila 450g -- 1.000
- PESSLOK   Miếng dán rốn giảm cân màu vàng kiểu cách độc đáo mới -- 1.000
- Mascara Focallure chuốt mi cong vút và dài hơn 24h nhanh khô chống thấm nước và mồ hôi lâu trôi 7.5g -- 0.795
- Xe Đẩy Gấp Gọn COBY SV-8, Khung Hợp Kim Nhôm, Ghế Nằm Ngồi, Đảo Chiều, Có Đệm Kèm Mái Che, BH 1 Năm -- 0.777
- Dầu xả khô dưỡng tóc tóc khô xơ không cần xả lại với nước Re-form Treatment Essence 15ml | Goldilocks -- 0.765
====
- Sữa Tăng Cân Cho Trẻ Em Appeton Weight Gain Junior hương Vanila 450g -- 1.000
- PESSLOK   Miếng dán rốn giảm cân màu vàng kiểu cách độc đáo mới -- 1.000
- Sirô Norikid Plus Cho Bé Ăn Ngon, Tăng Cân, Táo Bón, Tăng Cường Đề Kháng -- 0.788
- Cọ Trang Điểm Kẻ Chân Mày Kẻ Mắt Sợi Mảnh Tiện Lợi - Dụng Cụ Trang Điểm - COVERGIRL -- 0.788
- [GIÁ 582K LIVE AN PHƯƠNG] Trắng răng Smilee Extra, trắng răng từ 3-6 tone sau một liệu trì